# Digital Twin KPI Sanity Check (Interactive)

**Purpose:** Validate digital twin KPI predictions against generated sample data.

**Generated:** 2025-10-13


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider

# Deterministic seed
np.random.seed(42)

# Generate sample data
n_samples = 1000
timestamps = pd.date_range('2024-01-01', periods=n_samples, freq='H')
flight_ids = np.random.choice(['FL001','FL002','FL003','FL004','FL005'], n_samples)
actual_fuel = np.clip(np.random.normal(500, 50, n_samples), 300, 700)
prediction_error = np.random.normal(0, 30, n_samples)
predicted_fuel = actual_fuel + prediction_error

df = pd.DataFrame({
    'timestamp': timestamps,
    'flight_id': flight_ids,
    'h2_fuel_kg_actual': actual_fuel,
    'h2_fuel_kg_pred': predicted_fuel,
})
df.head()

## Interactive Accuracy Check

In [ ]:
def metrics(y_true, y_pred):
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)
    rmse = np.sqrt(np.mean((y_true - y_pred)**2))
    mae = np.mean(np.abs(y_true - y_pred))
    # R^2 with explicit formula
    ss_res = np.sum((y_true - y_pred)**2)
    ss_tot = np.sum((y_true - np.mean(y_true))**2)
    r2 = 1 - ss_res/ss_tot if ss_tot != 0 else float('nan')
    mape = np.mean(np.abs((y_true - y_pred) / np.clip(y_true, 1e-9, None))) * 100
    return rmse, mae, r2, mape

def analyze_fuel(threshold=5.0):
    fuel_pred = df['h2_fuel_kg_pred']
    fuel_act  = df['h2_fuel_kg_actual']
    rmse, mae, r2, mape = metrics(fuel_act, fuel_pred)
    
    print(f"RMSE: {rmse:.2f} kg  |  MAE: {mae:.2f} kg  |  R²: {r2:.4f}  |  MAPE: {mape:.2f}%")
    print("PASS" if mape < threshold else "FAIL")
    
    fig, ax = plt.subplots(figsize=(6,5))
    ax.scatter(fuel_act, fuel_pred, s=8, alpha=0.5)
    mn, mx = float(np.min(fuel_act)), float(np.max(fuel_act))
    ax.plot([mn, mx], [mn, mx])
    ax.set_xlabel('Actual H2 Fuel (kg)')
    ax.set_ylabel('Predicted H2 Fuel (kg)')
    ax.set_title('Predicted vs Actual')
    plt.show()

interact(analyze_fuel, threshold=FloatSlider(min=0, max=20, step=0.5, value=5.0, description='MAPE %'));

## Residuals

In [ ]:
residuals = df['h2_fuel_kg_pred'] - df['h2_fuel_kg_actual']
fig = plt.figure(figsize=(12,4))
plt.hist(residuals, bins=50)
plt.xlabel('Residual (kg)')
plt.ylabel('Count')
plt.title('Residual Distribution')
plt.show()

fig = plt.figure(figsize=(12,4))
plt.scatter(df['h2_fuel_kg_pred'], residuals, s=8, alpha=0.5)
plt.axhline(0)
plt.xlabel('Predicted H2 Fuel (kg)')
plt.ylabel('Residual (kg)')
plt.title('Residual vs Predicted')
plt.show()

## Daily MAPE Trend

In [ ]:
df['date'] = pd.to_datetime(df['timestamp']).dt.date
daily = df.groupby('date').apply(lambda x: np.mean(np.abs((x['h2_fuel_kg_actual']-x['h2_fuel_kg_pred'])/np.clip(x['h2_fuel_kg_actual'],1e-9,None)))*100)
fig = plt.figure(figsize=(10,4))
plt.plot(daily.index, daily.values)
plt.xticks(rotation=45)
plt.ylabel('MAPE (%)')
plt.title('Daily Prediction Error Trend')
plt.tight_layout()
plt.show()

## Summary

In [ ]:
rmse, mae, r2, mape = metrics(df['h2_fuel_kg_actual'], df['h2_fuel_kg_pred'])
summary = pd.DataFrame([{
    'Total Samples': len(df),
    'RMSE (kg)': rmse,
    'MAE (kg)': mae,
    'R2': r2,
    'MAPE (%)': mape,
    'Requirement': '<5% MAPE',
    'Status': 'PASS' if mape < 5.0 else 'FAIL'
}])
summary